In [1]:
%load_ext autoreload
%autoreload 2

import pygsti
from pygsti.modelpacks import smq1Q_XYI
import numpy as np
from pprint import pprint
import copy


def make_depolarized_dataset(modelpack, depol_level=0.01, max_max_len=128):
    ideal_model = modelpack.target_model()
    prep_fids = modelpack.prep_fiducials()
    meas_fids = modelpack.meas_fiducials()
    germs = modelpack.germs()
    max_lens = [2**p for p in range(1+int(np.log2(max_max_len)))]
    lsgst_circuit_lists = pygsti.circuits.create_lsgst_circuit_lists(ideal_model, prep_fids, meas_fids, germs, max_lens)
    all_circuits = lsgst_circuit_lists[-1]
    shots_per_circuit = 1000
    rng_state = np.random.default_rng(0)
    depol_model = ideal_model.depolarize(op_noise=depol_level)
    ds = pygsti.data.simulate_data(depol_model, all_circuits, shots_per_circuit, rand_state=rng_state)
    return ds


def corrupt_dataset(ds, prop_corrupt, rng=0):
    dsc = ds.copy_nonstatic()
    rng = np.random.default_rng(rng)
    num_circs = len(dsc)
    selected = rng.choice(np.arange(num_circs), size=int(num_circs*prop_corrupt), replace=False)
    circuits = list(dsc.keys())
    selected = [circuits[i] for i in selected]
    for c in selected:
        num_shots = dsc[c].total
        old_row   = dsc[c].to_dict()
        distn = rng.random(len(old_row))
        distn /= np.sum(distn)
        new_row = {k: num_shots * distn[i] for i,k in enumerate(old_row.keys())}
        dsc[c] = new_row
    dsc.comment  = 'corrupt'
    return dsc, selected


def cptp_gst(ds, fids, germs, target_model, final_objective: str, verbosity: int, mode='CPTPLND'):
    """
    In the context of this notebook, `ds` is produced by either make_depolarized_dataset or corrupt_dataset.
    final_objective can be 'tvd', 'chi2', or 'logl'.

    This function wraps up three steps of a GST pipeline.

    1. Construct a StandardGSTDesign based on (target_model, ds, fids, germs).
         * processor_spec is the value returned from target_model.create_processor_spec.
         * max_lens list is all powers of two that are <= the depth of the longest circuit in ds.
         * circuits in the design are filtered to only include circuits that appeared in ds.

    2. Construct a StandardGST protocol object based on (final_objective, mode, verbosity).
         * The gauge optimization suite is 'stdgaugeopt', minus the TPSpam optimization step.
         * objfn_builders, optimizer, and badfit_options are all set so the final 
           iteration's objective function is based on final_objective.

    3. Run GST with checkpointing turned off. 
        We dot NOT save the results to disk! The calling function is responsible for that.
    """
    max_exp = int(np.log2(np.max([len(c) for c in ds.keys()])))
    max_lens = [2**p for p in range(1 + max_exp)]
    prep_fids, meas_fids = fids

    target_model = target_model.copy()
    target_model.default_gauge_group = 'unitary'

    gos = pygsti.protocols.gst.GSTGaugeOptSuite.cast('stdgaugeopt')
    gop_params = gos.to_dictionary(target_model)
    # ^ a dict with one key, 'stdgaugeopt', whose corresponding value is a list of dicts.
    #   The internal dicts will indicate Frobenius-based losses for gates and SPAM,
    #   along with varying weights. Additional elements can be added to any one of these
    #   internal dicts to be passed to gaugeopt_to_target.
    gop_params['stdgaugeopt'] = gop_params['stdgaugeopt'][:-1]
    # ^ drop the 1-dimensional TPSpam gauge optimization step.

    exp_design = pygsti.protocols.StandardGSTDesign(
        target_model.create_processor_spec(),
        prep_fids, meas_fids, germs, max_lens,
        None,           # germ_length_limits
        None, 1, None,  # fidPairs, keepFraction, keepSeed
        True, True,     # include_lgst, nested_circuit_lists
        None,           # string_manipulation_rules
        None,           # op_label_aliases
        ds, 'drop', verbosity=verbosity
    )
    data = pygsti.protocols.ProtocolData(exp_design, ds)
    
    from pygsti.drivers.longsequence import _get_gst_builders, _get_optimizer, _get_badfit_options
    advanced_options = {'objective': final_objective}
    proto = pygsti.protocols.StandardGST(
        (mode,), gop_params, target_model, None, 
        objfn_builders    = _get_gst_builders(advanced_options),
        optimizer         = _get_optimizer(advanced_options, target_model),
        badfit_options    = _get_badfit_options(advanced_options),
        verbosity         = verbosity
    )
    results = proto.run(data, disable_checkpointing=True)
    return results



In [2]:
mp = smq1Q_XYI
target = mp.target_model()
fids = (mp.prep_fiducials(), mp.meas_fiducials())
germs = mp.germs()
ds = make_depolarized_dataset(mp, depol_level=0.01, max_max_len=128)

In [4]:
fit_mode = 'full TP'

results = cptp_gst(ds, fids, germs, target, 'logl', verbosity=1, mode=fit_mode)
results.estimates['fit-true-logl'] = results.estimates.pop(fit_mode)
tvd_res = cptp_gst(ds, fids, germs, target, 'tvd', verbosity=1, mode=fit_mode)
results.estimates['fit-true-tvd'] = tvd_res.estimates.pop(fit_mode)

dsc, selected = corrupt_dataset(ds, prop_corrupt=0.025)
logl_res = cptp_gst(dsc, fids, germs, target, 'logl', verbosity=1, mode=fit_mode)
results.estimates['fit-corrupt-logl'] = logl_res.estimates.pop(fit_mode)
tvd_res  = cptp_gst(dsc, fids, germs, target,  'tvd', verbosity=1, mode=fit_mode)
results.estimates['fit-corrupt-tvd'] =  tvd_res.estimates.pop(fit_mode)

--- Circuit Creation ---


/Users/rjmurr/Documents/pygsti-tvd/pygsti/pygsti/objectivefns/objectivefns.py:4483: RuntimeWarning: divide by zero encountered in divide
  p5over_lsvec = 0.5/lsvec


-- Std Practice:  [##################################################] 100.0%  (full TP) --
--- Circuit Creation ---


/Users/rjmurr/Documents/pygsti-tvd/pygsti/pygsti/objectivefns/objectivefns.py:4014: UserWarning: This derivative is discontinuous and does not return a full subgradient.
  _warnings.warn('This derivative is discontinuous and does not return a full subgradient.')


-- Std Practice:  [##################################################] 100.0%  (full TP) --
--- Circuit Creation ---


/Users/rjmurr/Documents/pygsti-tvd/pygsti/pygsti/objectivefns/objectivefns.py:4483: RuntimeWarning: divide by zero encountered in divide
  p5over_lsvec = 0.5/lsvec


-- Std Practice:  [##################################################] 100.0%  (full TP) --
--- Circuit Creation ---
-- Std Practice:  [##################################################] 100.0%  (full TP) --


In [7]:
moar_results = results.copy()
dsc.comment = 'corrupt'
moar_results.data.dataset = dsc
report = pygsti.report.construct_standard_report(
    {'eval-true'  : results,
     'eval-corrupt' : moar_results
    },
    advanced_options={'skip_sections': ('colorbox',)},
    title="GST Example Report", verbosity=2
)
# NOTE: can reach in and change the entry in report.switchboard.objfn_builder_modvi,
#       or anything else in the switchboard, according to my whims.
report.write_html("case0_reports_250217/exampleReport", auto_open=True, verbosity=1)


Running idle tomography
Computing switchable properties
Found standard clifford compilation from smq1Q_XYI
Found standard clifford compilation from smq1Q_XYI
Found standard clifford compilation from smq1Q_XYI
Found standard clifford compilation from smq1Q_XYI
Found standard clifford compilation from smq1Q_XYI
Found standard clifford compilation from smq1Q_XYI
Found standard clifford compilation from smq1Q_XYI
Found standard clifford compilation from smq1Q_XYI


/Users/rjmurr/Documents/pygsti-tvd/pygsti/pygsti/tools/optools.py:164: UserWarning:


            Input matrix is not PSD up to tolerance 1.8189894035458565e-12.
            We'll project out the bad eigenspaces to only work with the PSD part.
            

/Users/rjmurr/Documents/pygsti-tvd/pygsti/pygsti/tools/optools.py:172: UserWarning:


            The PSD part of the input matrix is not trace-1 up to tolerance 3.637978807091713e-12.
            Beware result!
            

/Users/rjmurr/Documents/pygsti-tvd/pygsti/pygsti/forwardsims/mapforwardsim.py:732: UserWarning:

Generating dense process matrix representations of circuits or gates 
can be inefficient and should be avoided for the purposes of forward 
simulation/calculation of circuit outcome probability distributions 
when using the MapForwardSimulator.



Statistical hypothesis tests did NOT find inconsistency between the data at 5.00% significance.
The data are INCONSISTENT at 5.00% significance.
  - Details:
    - The aggregate log-likelihood ratio test is significant at 167.04 standard deviations.
    - The aggregate log-likelihood ratio test standard deviations signficance threshold is 2.00
    - The number of sequences with data that is inconsistent is 21
    - The maximum SSTVD over all sequences is 0.78
    - The maximum SSTVD was observed for Qubit 0 ---|Gxpi2|-|Gxpi2|-|Gypi2|-|Gypi2|-|Gypi2|-|Gypi2|---

The data are INCONSISTENT at 5.00% significance.
  - Details:
    - The aggregate log-likelihood ratio test is significant at 167.04 standard deviations.
    - The aggregate log-likelihood ratio test standard deviations signficance threshold is 2.00
    - The number of sequences with data that is inconsistent is 21
    - The maximum SSTVD over all sequences is 0.78
    - The maximum SSTVD was observed for Qubit 0 ---|Gxpi2|-|Gxp